# Сведения о публикационной активности автора

In [67]:
import requests
import json
import pandas as pd

# Получение данных из API ORCID

In [68]:
url = "https://orcid.org/0000-0002-8520-7267/allWorks.json?sort=date&sortAsc=false"

print("Запрос к API ORCID...")
response = requests.get(url)

print(f"Статус ответа: {response.status_code}")

Запрос к API ORCID...
Статус ответа: 200


In [69]:
data = response.json()
print(f"Данные успешно загружены. Ключи верхнего уровня: {list(data.keys())}")
print(f"Всего работ: {data.get('totalGroups', 'не указано')}")

Данные успешно загружены. Ключи верхнего уровня: ['nextOffset', 'totalGroups', 'groups']
Всего работ: 711


# Изучение структуры данных

In [70]:
first_group = data['groups'][0]
print("Структура первой группы:")
print(f"Ключи группы: {list(first_group.keys())}")

Структура первой группы:
Ключи группы: ['activePutCode', 'defaultPutCode', 'groupId', 'activeVisibility', 'userVersionPresent', 'externalIdentifiers', 'works', 'featuredDisplayIndex']


In [71]:
first_work = first_group['works'][0]
print(f"\nКлючи work: {list(first_work.keys())}")


Ключи work: ['visibility', 'errors', 'publicationDate', 'putCode', 'shortDescription', 'url', 'journalTitle', 'languageCode', 'languageName', 'citation', 'countryCode', 'countryName', 'contributors', 'contributorsGroupedByOrcid', 'numberOfContributors', 'workExternalIdentifiers', 'source', 'sourceName', 'assertionOriginOrcid', 'assertionOriginClientId', 'assertionOriginName', 'title', 'subtitle', 'translatedTitle', 'workType', 'dateSortString', 'createdDate', 'lastModified', 'featuredDisplayIndex']


In [72]:
print("Структура title:")
print(json.dumps(first_work['title'], indent=2, ensure_ascii=False))

Структура title:
{
  "errors": [],
  "value": "Deep Unfolding of Iterative Precoding-Based Null-Space Expansion for Multi-User Massive MIMO in Time-Varying Channel",
  "required": true,
  "getRequiredMessage": null
}


In [73]:
print(f"\nГод публикации: {first_work['publicationDate']['year']}")
print(f"Название: {first_work['title']['value']}")


Год публикации: 2026
Название: Deep Unfolding of Iterative Precoding-Based Null-Space Expansion for Multi-User Massive MIMO in Time-Varying Channel


In [74]:
if 'workExternalIdentifiers' in first_work:
    identifiers = first_work['workExternalIdentifiers']
    for id in identifiers[:2]:
        print(f"Идентификатор {id['externalIdentifierType']['value']}: {id['externalIdentifierId']['value']}")

Идентификатор doi: 10.1109/TCOMM.2025.3642692


# Извлечение данных

In [75]:
def extract_work_data(work):
    """
    Извлекает год, название работы, doi и eid из одного описания работы (work):
    - Создает словарь с ключами.
    - Извлекает данные, проверяя наличие соответствующих ключей.
    - Проходит по индентификаторам и запоминает DOI и EID.
    """
    result = {
        'publication_year': None,
        'title': None,
        'doi': None,
        'eid': None
    }

    # Год публикации
    pub_date = work.get('publicationDate')
    if isinstance(pub_date, dict):
        result['publication_year'] = pub_date.get('year')

    # Название
    main_title = work.get('title')
    if isinstance(main_title, dict):
        result['title'] = main_title.get('value')

    # Идентификаторы (DOI и EID)
    identifiers = work.get('workExternalIdentifiers', [])
    if isinstance(identifiers, list):
        for id in identifiers:
            if not isinstance(id, dict):
                continue

    id_type_dict = id.get('externalIdentifierType', {})
    id_type = id_type_dict.get('value') if isinstance(id_type_dict, dict) else ''

    id_value_dict = id.get('externalIdentifierId', {})
    id_value = id_value_dict.get('value') if isinstance(id_value_dict, dict) else None


    if id_type == 'doi' and id_value:
        result['doi'] = id_value
    elif id_type == 'eid' and id_value:
        result['eid'] = id_value

    return result

# Все работы. Сбор данных

In [76]:
# Список для хранения всех записей
all_records = []

total_groups = len(data['groups'])

for group_idx, group in enumerate(data['groups']):
    # Отслеживание прогресса по 100 групп
    if group_idx % 100 == 0:
        print(f"Обработано {group_idx} из {total_groups} групп")

    # Обрабатка всех works в группе
    for work in group.get('works', []):
        record = extract_work_data(work)

        # Добавляем, если указан год публикации
        if record['publication_year']:
            all_records.append(record)

print(f"Обработка завершена! Собрано {len(all_records)} записей")

Обработано 0 из 711 групп
Обработано 100 из 711 групп
Обработано 200 из 711 групп
Обработано 300 из 711 групп
Обработано 400 из 711 групп
Обработано 500 из 711 групп
Обработано 600 из 711 групп
Обработано 700 из 711 групп
Обработка завершена! Собрано 1191 записей


In [77]:
# DataFrame с данными
df = pd.DataFrame(all_records)

df.head()

,publication_year,title,doi,eid
0,2026,Deep Unfolding of Iterative Precoding-Based Nu...,10.1109/TCOMM.2025.3642692,None
1,2026,Resource Allocation Schemes for Scalable Panel...,10.1109/OJVT.2026.3652908,None
2,2025,On Deep Learning Hybrid Architectures for MIMO...,10.3390/electronics14234692,None
3,2025,On Deep Learning Hybrid Architectures for MIMO...,10.3390/electronics14234692,None
4,2025,On Deep Learning Hybrid Architectures for MIMO...,10.3390/electronics14234692,None


In [78]:
# DataFrame, очищенный от дубликатов (исключаем записи о работах, встречающиеся в более чем 1 works)
df = df.drop_duplicates(subset=['title', 'publication_year'])
df_unique = df.reset_index(drop=True)

print(f"После удаления дубликатов: {len(df_unique)} уникальных работ")
df_unique.head()

После удаления дубликатов: 765 уникальных работ


,publication_year,title,doi,eid
0,2026,Deep Unfolding of Iterative Precoding-Based Nu...,10.1109/TCOMM.2025.3642692,None
1,2026,Resource Allocation Schemes for Scalable Panel...,10.1109/OJVT.2026.3652908,None
2,2025,On Deep Learning Hybrid Architectures for MIMO...,10.3390/electronics14234692,None
3,2025,Data-Oriented Channel Knowledge Map IoT Transm...,None,None
4,2025,Newmann Series-Based Precoding Weight Design f...,None,None


In [79]:
csv_name = 'orcid_year_title_doi_eid.csv'
df_unique.to_csv(csv_name, index=False, encoding='utf-8')

print(f"DataFrame сохранен в csv: {csv_name}")

DataFrame сохранен в csv: orcid_year_title_doi_eid.csv


In [80]:
json_name = 'orcid_year_title_doi_eid.json'
df_unique.to_json(json_name, orient='records', force_ascii=False, indent=2)

print(f"Данные сохранены в json: {json_name}")

Данные сохранены в json: orcid_year_title_doi_eid.json


**Сделано:**

- Полученны данные из API.
- Извлечен массив с требуемыми полями (df - содержит все записи; df_unique - только уникальные для каждой работы и соотвествуюего года публикации).
- Данные сохранены в файлы для передачи (в двух форматах: csv и json).

Запрашиваемый массив содержит сведения о 765 уникальных работах.

**Пояснение к полученным данным:**

В API ORCID представлено 711 групп (groups) - это количество *уникальных публикаций* в системе.

В результате обработки получено 765 уникальных записей - это количество *уникальных названий работ*.

Разница в 54 записи объясняется особенностями агрегации данных из разных источников:
- Одна и та же научная публикация может иметь разные варианты названия в различных базах данных (Crossref, Scopus, IEEE, и т.д.).
- При удалении дубликатов по полю "год  + название" такие варианты считаются разными записями.

В данной задаче дубликаты удалялись по комбинации "год + название", так как:
- эти поля присутствуют у всех записей (записи, для которых не указан год, в конечный список не были включены);
- задача требовала извлечь названия работ;
- идентификаторы DOI, EID есть не у всех публикаций => данные будут неполными.

Таким образом, текущий подход обеспечивает максимальную полноту данных
при сохранении уникальности записей.